In [2]:
import pandas as pd

In [4]:
dataset = pd.read_csv("/Users/ykasapien/Desktop/SPAM text message 20170820 - Data.csv")

In [ ]:
dataset.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
print(dataset.groupby('Category').count())

          Message
Category         
ham          4825
spam          747


**One-Hot Encoder**

In [6]:
vector_labels = pd.get_dummies(dataset['Category'])

In [7]:
type(vector_labels)

pandas.core.frame.DataFrame

In [ ]:
#merge dataset and one-hot encoder vector
dataset = pd.merge(
    left=dataset,
    right=vector_labels,
    left_index=True,
    right_index=True,
)

In [ ]:
dataset.head()

,Category,Message,ham,spam
0,ham,"Go until jurong point, crazy.. Available only ...",1,0
1,ham,Ok lar... Joking wif u oni...,1,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,ham,U dun say so early hor... U c already then say...,1,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",1,0


**Message Preparing**

In [ ]:
type(dataset['Message'][0])

str

In [ ]:
#to del punctuation marks
import re
#for Lemmatizer and stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lm = WordNetLemmatizer()

In [ ]:
list_of_prepared_messages = []
for i in range(0, len(dataset)):
  #to del punctuation marks
  message_preparation = re.sub('[^a-zA-Z]', ' ', dataset['Message'][i])
  #to lower case and change to list of words
  message_preparation = message_preparation.lower().split()
  #for Lemmatizer and skip stopwords
  message_preparation = [lm.lemmatize(word) for word in message_preparation if not word in stopwords.words('english')]
  #join again to one string
  message_preparation = ' '.join(message_preparation)
  list_of_prepared_messages.append(message_preparation)

In [ ]:
#change to series
prepared_messages_series = pd.Series(list_of_prepared_messages, name = 'prepared_messages')

In [ ]:
prepared_messages_series[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [ ]:
#merge dataset and prepared_messages
dataset = pd.merge(
    left=dataset,
    right=prepared_messages_series,
    left_index=True,
    right_index=True,
)

In [ ]:
dataset.head()

,Category,Message,ham,spam,prepared_messages
0,ham,"Go until jurong point, crazy.. Available only ...",1,0,go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,1,0,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,1,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,1,0,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",1,0,nah think go usf life around though


**Using the built-in TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
trasformed_features = vectorizer.fit_transform(dataset["prepared_messages"])

In [ ]:
vectors_dense = trasformed_features.todense()
dense_list = vectors_dense.tolist()

In [ ]:
names = vectorizer.get_feature_names()

In [ ]:
tfidf_df = pd.DataFrame(dense_list, columns=names)

In [ ]:
tfidf_df